In [21]:
# Set the folder paths and dataset CSV path (customize these according to your setup)
image_folder = 'C:\\Users\\Jeffan\\Downloads\\MVSA-Single\\MVSA_Single\\data'
dataset_csv = 'C:\\Users\\Jeffan\\Documents\\GitHub\\CS16-1-Sentiment-Analysis\\Data Image\\output_single.csv'
train_folder = 'C:\\Users\\Jeffan\\Downloads\\MVSA-Single\\MVSA_Single\\train'
val_folder = 'C:\\Users\\Jeffan\\Downloads\\MVSA-Single\\MVSA_Single\\val'
test_folder = 'C:\\Users\\Jeffan\\Downloads\\MVSA-Single\\MVSA_Single\\test'

In [22]:
ids = []
texts = []
images = []
with open('C:\\Users\\Jeffan\\Downloads\\MVSA-Single\\MVSA_Single\\labelResultAll.txt', 'r', encoding='ISO-8859-1') as file:
    for line in file:
        text_list = line.replace('\n','').split('	')
        text_label = text_list[1].split(',')[0]
        image_label = text_list[1].split(',')[1]
        if not ((text_label=='positive' and image_label=='negative') or (text_label=='positive' and image_label=='negative')):
            ids.append(text_list[0])
            texts.append(text_label)
            images.append(image_label)
ids.pop(0)
texts.pop(0)
images.pop(0)

'image'

In [23]:
import pandas as pd
dataset = {'ID':ids, 'Text':texts, 'Image':images}
df = pd.DataFrame(dataset)
df.to_csv('output_single.csv', index=False)
df

,ID,Text,Image
0,1,neutral,positive
1,2,neutral,positive
2,3,neutral,positive
3,4,positive,positive
4,5,positive,positive
...,...,...,...
4786,5125,neutral,positive
4787,5126,positive,neutral
4788,5127,positive,positive
4789,5128,neutral,positive


In [24]:
import random
import shutil

def oversample_minority_classes(df, folder):
    class_counts = df['Image'].value_counts()
    max_count = class_counts.max()

    for cls, count in class_counts.items():
        if count < max_count:
            cls_df = df[df['Image'] == cls]
            num_samples_needed = round(max_count*0.7) - count

            for i in range(num_samples_needed):
                row = cls_df.sample().iloc[0]
                original_image_path = os.path.join(folder, row['Image'], f"{row['ID']}.jpg")
                new_image_id = f"{row['ID']}_oversampled_{i}"
                new_image_path = os.path.join(folder, row['Image'], f"{new_image_id}.jpg")
                shutil.copy(original_image_path, new_image_path)
                new_row = row.copy()
                new_row['ID'] = new_image_id
                df = df.append(new_row, ignore_index=True)

    return df

In [25]:
import os
import shutil
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline

def split_balanced_images(image_folder, dataset_csv, train_folder, val_folder, test_folder, ratio=(8, 1, 1)):
    # Read the dataset CSV file
    df = pd.read_csv(dataset_csv)

    # Shuffle the dataset
    df = df.sample(frac=1).reset_index(drop=True)

    # Create the output folders if they don't exist
    os.makedirs(train_folder, exist_ok=True)
    os.makedirs(val_folder, exist_ok=True)
    os.makedirs(test_folder, exist_ok=True)

    unique_classes = df['Image'].unique()

    # Create class subfolders in train, val, and test folders
    for cls in unique_classes:
        os.makedirs(os.path.join(train_folder, cls), exist_ok=True)
        os.makedirs(os.path.join(val_folder, cls), exist_ok=True)
        os.makedirs(os.path.join(test_folder, cls), exist_ok=True)
    
    train_val_ratio = (ratio[0] + ratio[1]) / sum(ratio)
    train_ratio = ratio[0] / (ratio[0] + ratio[1])

    train_val_df, test_df = train_test_split(df, test_size=1 - train_val_ratio, stratify=df['Image'], random_state=42)
    train_df, val_df = train_test_split(train_val_df, test_size=1 - train_ratio, stratify=train_val_df['Image'], random_state=42)

    # Define the undersampling strategy
    undersample = RandomUnderSampler(sampling_strategy='majority')

    # Encode class labels
    encoder = LabelEncoder()
    train_val_df['Image'] = encoder.fit_transform(train_val_df['Image'])
    train_df['Image'] = encoder.transform(train_df['Image'])
    val_df['Image'] = encoder.transform(val_df['Image'])

    # Undersample the training set
    X_train = train_df.drop(columns=['Image'])
    y_train = train_df['Image']
    X_train_resampled, y_train_resampled = undersample.fit_resample(X_train, y_train)
    train_df_resampled = pd.concat([X_train_resampled, y_train_resampled], axis=1)

    # Undersample the validation set
    X_val = val_df.drop(columns=['Image'])
    y_val = val_df['Image']
    X_val_resampled, y_val_resampled = undersample.fit_resample(X_val, y_val)
    val_df_resampled = pd.concat([X_val_resampled, y_val_resampled], axis=1)

    # Decode class labels
    train_df_resampled['Image'] = encoder.inverse_transform(train_df_resampled['Image'])
    val_df_resampled['Image'] = encoder.inverse_transform(val_df_resampled['Image'])

    # Move the images to the corresponding class subfolders
    for _, row in train_df_resampled.iterrows():
        shutil.copy(os.path.join(image_folder, f"{row['ID']}.jpg"), os.path.join(train_folder, row['Image'], f"{row['ID']}.jpg"))
    for _, row in val_df_resampled.iterrows():
        shutil.copy(os.path.join(image_folder, f"{row['ID']}.jpg"), os.path.join(val_folder, row['Image'], f"{row['ID']}.jpg"))
    for _, row in test_df.iterrows():
        shutil.copy(os.path.join(image_folder, f"{row['ID']}.jpg"), os.path.join(test_folder, row['Image'], f"{row['ID']}.jpg"))

# Split the images
split_balanced_images(image_folder, dataset_csv, train_folder, val_folder, test_folder)

In [27]:
from keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

train_generator = train_datagen.flow_from_directory(
    directory= train_folder,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    shuffle=True
)

ImportError: cannot import name 'type_spec_registry' from 'tensorflow.python.framework' (c:\Users\Jeffan\.conda\envs\capstone2\lib\site-packages\tensorflow\python\framework\__init__.py)

In [ ]:
pd.Index(train_generator.classes).value_counts()

2    2166
0    1516
1    1516
dtype: int64